### Import required frameworks and libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

/var/folders/bf/f89y356d061dpv04rhvw5mpw0000gn/T/ipykernel_29604/2632832034.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('Hallucination-Dataset-400-Samples.csv')

In [3]:
# Data cleaning
df['Context'] = df['Context'].astype(str)
df['Question'] = df['Question'].astype(str)
df['Answer'] = df['Answer'].astype(str)

In [4]:
df['Combined'] = df['Context'] + ' ' + df['Question']

In [5]:
# Tokenize text for Combined column
max_words_combined = 1000
tokenizer_combined = Tokenizer(num_words=max_words_combined, oov_token='<OOV>')
tokenizer_combined.fit_on_texts(df['Combined'])
X_combined = tokenizer_combined.texts_to_sequences(df['Combined'])
X_combined = pad_sequences(X_combined, maxlen=100, padding='post', truncating='post')

In [6]:
# Tokenize text for Answer column
max_words_answer = 500
tokenizer_answer = Tokenizer(num_words=max_words_answer, oov_token='<OOV>')
tokenizer_answer.fit_on_texts(df['Answer'])
X_answer = tokenizer_answer.texts_to_sequences(df['Answer'])
X_answer = pad_sequences(X_answer, maxlen=50, padding='post', truncating='post')

In [7]:
# Concatenate the tokenized sequences
X_combined = pad_sequences(X_combined, maxlen=100, padding='post', truncating='post')
X_answer = pad_sequences(X_answer, maxlen=50, padding='post', truncating='post')
X = np.concatenate([X_combined, X_answer], axis=1)

In [8]:
# Encode hallucination labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Hallucination'])

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Build the neural network model
model = Sequential()
model.add(Embedding(input_dim=max_words_combined + max_words_answer, output_dim=100, input_length=150))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.3)

Epoch 1/5
7/7 [==============================] - 3s 259ms/step - loss: 0.6867 - accuracy: 0.6071 - val_loss: 0.6800 - val_accuracy: 0.5773
Epoch 2/5
7/7 [==============================] - 1s 188ms/step - loss: 0.6699 - accuracy: 0.6027 - val_loss: 0.6742 - val_accuracy: 0.5773
Epoch 3/5
7/7 [==============================] - 1s 199ms/step - loss: 0.6415 - accuracy: 0.6027 - val_loss: 0.6328 - val_accuracy: 0.5979
Epoch 4/5
7/7 [==============================] - 1s 211ms/step - loss: 0.5320 - accuracy: 0.7455 - val_loss: 0.5806 - val_accuracy: 0.6495
Epoch 5/5
7/7 [==============================] - 1s 218ms/step - loss: 0.3671 - accuracy: 0.8795 - val_loss: 0.6586 - val_accuracy: 0.6392


In [12]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

3/3 [==============================] - 1s 53ms/step


In [13]:
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.8148148148148148
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.83      0.77        30
           1       0.89      0.80      0.85        51

    accuracy                           0.81        81
   macro avg       0.80      0.82      0.81        81
weighted avg       0.83      0.81      0.82        81



In [14]:
y_pred[:10]

array([[1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1]])

In [15]:
y_test[:10]

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 0])